## Setup

### Imports

In [1]:
import os
import json
import glob
import torch
import re
import einops
import pandas as pd
from functools import partial
from torch import Tensor
from torchtyping import TensorType as TT


import plotly.express as px

from utils.data_utils import generate_data_and_caches
from utils.data_processing import (
    load_edge_scores_into_dictionary,
)
from utils.visualization import plot_attention_heads, imshow_p
from utils.backup_analysis import (
    load_model,
    run_iteration,
    process_backup_results,
    get_past_nmhs_for_checkpoints,
    plot_top_heads
)

### Parameters

In [2]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
BASE_MODEL = "pythia-160m"
VARIANT = 'pythia-160m-alldropout'
CACHE = "model_cache"
IOI_DATASET_SIZE = 70

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

torch.set_grad_enabled(False)

### Functions

## Experiments

### Experiment Parameters

In [3]:
TASK = 'ioi'
PERFORMANCE_METRIC = 'logit_diff'
BASE_MODEL = "pythia-160m"
VARIANT = "EleutherAI/pythia-160m-seed1"
MODEL_SHORTNAME = BASE_MODEL if not VARIANT else VARIANT[11:]
CACHE = "model_cache"
IOI_DATASET_SIZE = 70
COPY_SCORE_THRESHOLD = 75.0

### Circuit Data

In [4]:
folder_path = f'results/graphs/{MODEL_SHORTNAME}/{TASK}'
df = load_edge_scores_into_dictionary(folder_path)

# filter everything before 1000 steps
df = df[df['checkpoint'] >= 1000]

df[['source', 'target']] = df['edge'].str.split('->', expand=True)
len(df['target'].unique())

Processing file 1/140: results/graphs/pythia-160m-seed1/ioi/57000.json
Processing file 2/140: results/graphs/pythia-160m-seed1/ioi/141000.json
Processing file 3/140: results/graphs/pythia-160m-seed1/ioi/95000.json
Processing file 4/140: results/graphs/pythia-160m-seed1/ioi/107000.json
Processing file 5/140: results/graphs/pythia-160m-seed1/ioi/34000.json
Processing file 6/140: results/graphs/pythia-160m-seed1/ioi/6000.json
Processing file 7/140: results/graphs/pythia-160m-seed1/ioi/37000.json
Processing file 8/140: results/graphs/pythia-160m-seed1/ioi/39000.json
Processing file 9/140: results/graphs/pythia-160m-seed1/ioi/104000.json
Processing file 10/140: results/graphs/pythia-160m-seed1/ioi/59000.json
Processing file 11/140: results/graphs/pythia-160m-seed1/ioi/67000.json
Processing file 12/140: results/graphs/pythia-160m-seed1/ioi/111000.json
Processing file 13/140: results/graphs/pythia-160m-seed1/ioi/76000.json
Processing file 14/140: results/graphs/pythia-160m-seed1/ioi/5000.json

445

### Dataset Setup

In [5]:
initial_model = load_model(BASE_MODEL, VARIANT, 143000, CACHE, device)
size=70
ioi_dataset, abc_dataset = generate_data_and_caches(initial_model, size, verbose=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-seed1 at step143000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer


In [6]:
# imshow_p(
#     per_head_ablated_logit_diffs,
#     title="Headwise logit diff contribution, post NMH KO",
#     labels={"x": "Head", "y": "Layer", "color": "Logit diff attribution"},
#     #coloraxis=dict(colorbar_ticksuffix = "%"),
#     border=True,
#     width=600,
#     margin={"r": 100, "l": 100}
# )

### Run Experiment

In [10]:
experiment_metrics = dict()
# create folder
os.makedirs(f'results/backup/{MODEL_SHORTNAME}', exist_ok=True)

for checkpoint in range(4000, 144000, 1000):

    experiment_metrics = run_iteration(
        BASE_MODEL, VARIANT, df, checkpoint=checkpoint, dataset=ioi_dataset, experiment_metrics=experiment_metrics, 
        threshold=COPY_SCORE_THRESHOLD
    )
    experiment_metrics = process_backup_results(df, checkpoint, experiment_metrics)

    # save to file, using pytorch format
    torch.save(experiment_metrics, f'results/backup/{MODEL_SHORTNAME}/nmh_backup_metrics.pt')

Loaded model EleutherAI/pythia-160m-seed1 at step4000; now loading into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer
Checkpoint 4000:
Heads ablated:            []
Original logit diff:      0.4883192480
Post ablation logit diff: 0.4883192480
Logit diff % change:      0.00%
Loaded model EleutherAI/pythia-160m-seed1 at step5000; now loading into HookedTransformer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 6.5 (sign=1) : Top 5 accuracy: 100.0%
Checkpoint 5000:
Heads ablated:            [(6, 5)]
Original logit diff:      1.3061420918
Post ablation logit diff: 1.2755876780
Logit diff % change:      -2.34%
Loaded model EleutherAI/pythia-160m-seed1 at step6000; now loading into HookedTransformer


...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-seed1 at step142000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 6.5 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 10.9 (sign=1) : Top 5 accuracy: 5.238095238095238%
Copy circuit for head 9.11 (sign=1) : Top 5 accuracy: 7.142857142857142%
Copy circuit for head 9.10 (sign=1) : Top 5 accuracy: 3.8095238095238098%
Copy circuit for head 9.9 (sign=1) : Top 5 accuracy: 0.9523809523809524%
Copy circuit for head 7.7 (sign=1) : Top 5 accuracy: 4.761904761904762%
Copy circuit for head 7.9 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 9.0 (sign=1) : Top 5 accuracy: 4.285714285714286%
Copy circuit for head 9.1 (sign=1) : Top 5 accuracy: 9.047619047619047%
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 62.857142857142854%
Checkpoint 142000:
Heads ablated:            [(6, 5), (7, 9)]
Original logit diff:      3.8105905056
Post ablation logit diff: 4.3644423485
Logit d

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m-seed1 at step143000; now loading into HookedTransformer
Loaded pretrained model pythia-160m into HookedTransformer
Copy circuit for head 6.5 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 10.9 (sign=1) : Top 5 accuracy: 5.714285714285714%
Copy circuit for head 10.11 (sign=1) : Top 5 accuracy: 0.4761904761904762%
Copy circuit for head 9.10 (sign=1) : Top 5 accuracy: 2.857142857142857%
Copy circuit for head 7.7 (sign=1) : Top 5 accuracy: 3.8095238095238098%
Copy circuit for head 7.9 (sign=1) : Top 5 accuracy: 100.0%
Copy circuit for head 9.0 (sign=1) : Top 5 accuracy: 4.761904761904762%
Copy circuit for head 9.1 (sign=1) : Top 5 accuracy: 9.047619047619047%
Copy circuit for head 9.4 (sign=1) : Top 5 accuracy: 60.0%
Checkpoint 143000:
Heads ablated:            [(6, 5), (7, 9)]
Original logit diff:      3.9022033215
Post ablation logit diff: 4.2479395866
Logit diff % change:      8.86%


: 

In [9]:
experiment_metrics.keys()

dict_keys([4000, 5000, 6000, 7000])

## View Results

#### Pythia 160m

In [25]:
MODEL_TO_VIEW = "pythia-160m-hiddendropout"

In [26]:
experiment_metrics = torch.load(f'results/backup/{MODEL_TO_VIEW}/nmh_backup_metrics.pt')

In [27]:
experiment_metrics[4000].keys()

dict_keys(['logit_diff', 'per_head_logit_diffs', 'ablation_targets', 'ablated_logit_diff', 'per_head_ablated_logit_diffs', 'per_head_logit_diff_delta', 'in_circuit_head_delta', 'outside_circuit_head_delta', 'summed_in_circuit_head_delta', 'summed_outside_circuit_head_delta', 'summed_total_head_delta'])

In [28]:
summed_in_circuit_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_in_circuit_head_delta"] for checkpoint in experiment_metrics.keys()}
summed_outside_circuit_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_outside_circuit_head_delta"] for checkpoint in experiment_metrics.keys()}
summed_total_head_deltas = {checkpoint: experiment_metrics[checkpoint]["summed_total_head_delta"] for checkpoint in experiment_metrics.keys()}
per_head_logit_diff_deltas = {checkpoint: experiment_metrics[checkpoint]["per_head_logit_diff_delta"] for checkpoint in experiment_metrics.keys()}
total_logit_diff_deltas = {checkpoint: experiment_metrics[checkpoint]['ablated_logit_diff'] - experiment_metrics[checkpoint]['logit_diff'] for checkpoint in experiment_metrics.keys()}

for checkpoint in experiment_metrics.keys():
    # divide by total original logit diff
    summed_in_circuit_head_deltas[checkpoint] = summed_in_circuit_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    summed_outside_circuit_head_deltas[checkpoint] = summed_outside_circuit_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    summed_total_head_deltas[checkpoint] = summed_total_head_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    per_head_logit_diff_deltas[checkpoint] = per_head_logit_diff_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]
    total_logit_diff_deltas[checkpoint] = total_logit_diff_deltas[checkpoint] / experiment_metrics[checkpoint]["logit_diff"]

In [29]:
# plot summed_in_circuit_head_deltas with plotly express
fig = px.line(
    x=list(summed_in_circuit_head_deltas.keys()), 
    y=list(summed_in_circuit_head_deltas.values()), 
    title=f"Summed Post-NMH-Ablation In-Circuit Head Logit Diff Change Over Time ({MODEL_TO_VIEW})",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'} 
)
fig.show()


In [30]:
# plot summed_outside_circuit_head_deltas
fig = px.line(
    x=list(summed_outside_circuit_head_deltas.keys()), 
    y=list(summed_outside_circuit_head_deltas.values()), 
    title=f"Summed Post-NMH-Ablation Outside-Circuit Head Attribution Change ({MODEL_TO_VIEW})",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'} 
)
fig.show()

In [31]:
# plot total_head_deltas
fig = px.line(
    x=list(summed_total_head_deltas.keys()), 
    y=list(summed_total_head_deltas.values()), 
    title=f"Summed Total Post-NMH-Ablation Head Attribution Change ({MODEL_TO_VIEW})",
    labels={'x': 'Checkpoint', 'y': 'Change as % of original logit diff'}
)

fig.show()

In [32]:
cumulative_nmhs, checkpoint_nmhs = get_past_nmhs_for_checkpoints(experiment_metrics)

In [33]:
top_backup_heads = plot_top_heads(model_name=MODEL_TO_VIEW, checkpoint_dict=per_head_logit_diff_deltas, cumulative_nmhs=cumulative_nmhs, top_k_per_checkpoint=10)

In [57]:
#per_head_logit_diff_deltas

imshow_p(
    experiment_metrics[143000]['per_head_logit_diff_delta'], #[143000],
    title="Headwise logit diff contribution, post NMH KO",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff attribution"},
    #coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

In [58]:
experiment_metrics[143000].keys()

dict_keys(['logit_diff', 'per_head_logit_diffs', 'ablation_targets', 'ablated_logit_diff', 'per_head_ablated_logit_diffs', 'per_head_logit_diff_delta', 'in_circuit_head_delta', 'outside_circuit_head_delta', 'summed_in_circuit_head_delta', 'summed_outside_circuit_head_delta', 'summed_total_head_delta'])

In [59]:
top_backup_heads[top_backup_heads['Previous NMH']==True].head(50)

,Checkpoint,Layer-Head,Layer,Head,Value,Previous NMH,Checkpoint_sum,Value_sum,Previous NMH_sum,Top K
72,20000,Layer 9-Head 4,9,4,0.002460,True,3135000,0.334368,38,True
97,25000,Layer 9-Head 4,9,4,0.006947,True,3135000,0.334368,38,True
121,30000,Layer 9-Head 4,9,4,0.006404,True,3135000,0.334368,38,True
149,35000,Layer 9-Head 4,9,4,0.014621,True,3135000,0.334368,38,True
156,37000,Layer 9-Head 4,9,4,0.013440,True,3135000,0.334368,38,True
171,40000,Layer 9-Head 4,9,4,0.010845,True,3135000,0.334368,38,True
197,45000,Layer 9-Head 4,9,4,0.035425,True,3135000,0.334368,38,True
202,46000,Layer 9-Head 4,9,4,0.033058,True,3135000,0.334368,38,True
207,47000,Layer 9-Head 4,9,4,0.014968,True,3135000,0.334368,38,True
212,48000,Layer 9-Head 4,9,4,0.008163,True,3135000,0.334368,38,True


In [60]:
checkpoint_nmhs

{4000: set(),
 5000: set(),
 6000: set(),
 7000: set(),
 8000: {(8, 2)},
 9000: set(),
 10000: {(8, 1), (8, 10)},
 11000: {(10, 7)},
 12000: {(8, 2), (10, 7)},
 13000: {(10, 7)},
 14000: {(8, 2), (10, 7)},
 15000: {(8, 2), (10, 7)},
 16000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 17000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 18000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 19000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 20000: {(8, 2), (10, 7)},
 21000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 22000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 23000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 24000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 25000: {(8, 2), (10, 7)},
 26000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 27000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 28000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 29000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 30000: {(8, 2), (10, 7)},
 31000: {(8, 1), (8, 2), (9, 4), (10, 7)},
 32000: {(8, 1), (8

In [61]:
cumulative_nmhs

{4000: set(),
 5000: set(),
 6000: set(),
 7000: set(),
 8000: {(8, 2)},
 9000: {(8, 2)},
 10000: {(8, 1), (8, 2), (8, 10)},
 11000: {(8, 1), (8, 2), (8, 10), (10, 7)},
 12000: {(8, 1), (8, 2), (8, 10), (10, 7)},
 13000: {(8, 1), (8, 2), (8, 10), (10, 7)},
 14000: {(8, 1), (8, 2), (8, 10), (10, 7)},
 15000: {(8, 1), (8, 2), (8, 10), (10, 7)},
 16000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 17000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 18000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 19000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 20000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 21000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 22000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 23000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 24000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 25000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 26000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 27000: {(8, 1), (8, 2), (8, 10), (9, 4), (10, 7)},
 28000: {(8, 1), (8, 2), (8, 10

In [62]:
# plot number of nmhs over time
fig = px.line(
    x=list(checkpoint_nmhs.keys()), 
    y=list([len(heads) for heads in checkpoint_nmhs.values()]), 
    title=f"Number of NMHs Over Time ({MODEL_TO_VIEW})",
    labels={'x': 'Checkpoint', 'y': 'Number of NMHs'}
)
fig.show()